In [1]:
import cProfile
import io
import os
from venv import create

import requests
import pstats

import numpy as np
from laspy import LasData

import functions
from scipy.spatial import cKDTree 
from scipy.interpolate import griddata
import startinpy
import rasterio


In [2]:
import laspy
import numpy as np

def extract_vegetation_points(LasData, ndvi_threshold=0.1):
    """
    Extract vegetation points based on classification and NDVI threshold.
    
    Parameters:
    - LasData (laspy.LasData): Input point cloud data.
    - ndvi_threshold (float): The NDVI threshold to classify vegetation.
    
    Returns:
    - veg_points (laspy.LasData): Filtered vegetation points as a new LasData object.
    """
    
    # Filter points based on classification (vegetation-related classes)
    possible_vegetation_points = LasData[(LasData.classification == 1) |  # Unclassified
                                         (LasData.classification == 3) |  # Low vegetation
                                         (LasData.classification == 4) |  # Medium vegetation
                                         (LasData.classification == 5)]   # High vegetation

    # Calculate NDVI
    red = possible_vegetation_points.red
    nir = possible_vegetation_points.nir
    ndvi = (nir.astype(float) - red) / (nir + red)

    # Filter the points whose NDVI is greater than the threshold
    veg_points = possible_vegetation_points[ndvi > ndvi_threshold]
    
    # EDIT THIS, ONLY FOR THE TILE WITH ISSUES (25GN224LAZ)
    heights =veg_points.z
    min_height = heights.min()
    
    # Filter out points with heights between the minimum height and 1.5 meters
    filtered_veg_points = veg_points[(heights <= min_height) | (heights > 1.5)]

    return filtered_veg_points

def save_vegetation_points_as_las(LasData, output_file, veg_points, ndvi_threshold=0.1):
    """
    Extract vegetation points and save them as a new LAS file.
    
    Parameters:
    - LasData (laspy.LasData): Input point cloud data.
    - output_file (str): Path to the output LAS file.
    - ndvi_threshold (float): NDVI threshold for vegetation points.
    
    Returns:
    - None
    """
    # Extract vegetation points
    
    # Check if any points were filtered
    if len(veg_points) == 0:
        print("No vegetation points found with the given NDVI threshold.")
        return
    
    # Create a new LasData object for the vegetation points
    veg_las_data = laspy.LasData(LasData.header)
    veg_las_data.points = veg_points.points  # Assign the filtered vegetation points

    # Save the vegetation points to a new LAS file
    with laspy.open(output_file, mode="w", header=veg_las_data.header) as writer:
        writer.write_points(veg_las_data.points)
    
    print(f"Vegetation points saved to {output_file}")


file_path = "C:/Geomatics/data/25GN2_24.LAZ"
output_las = "data/output_25gn224.las"

In [3]:
import numpy as np
import laspy

def extract_lasData(LasData, ndvi_threshold=0.1):
    red = LasData.red
    nir = LasData.nir
    ndvi = (nir.astype(float) - red) / (nir + red)

    # Add NDVI as a new dimension/attribute to all points
    if 'NDVI' not in LasData.point_format.dimension_names:
        # Create an array to store NDVI values for all points
        ndvi_dim = np.zeros(LasData.header.point_count, dtype=np.float32)
        ndvi_dim = ndvi  # Assign NDVI values for all points

        # Add this new dimension to LasData
        LasData.add_extra_dim(laspy.ExtraBytesParams(name="NDVI", type=np.float32))
        LasData['NDVI'] = ndvi_dim  # Store NDVI for all points

    # Filter points based on classification (vegetation-related classes)
    possible_vegetation_points = LasData[(LasData.classification == 1) |  # Unclassified
                                         (LasData.classification == 3) |  # Low vegetation
                                         (LasData.classification == 4) |  # Medium vegetation
                                         (LasData.classification == 5)]   # High vegetation


    return LasData

In [4]:
with laspy.open(file_path, laz_backend=laspy.LazBackend.Lazrs) as las_file:
    las_data = las_file.read()

veg_points = extract_vegetation_points(las_data, ndvi_threshold=0.0)

In [6]:
las_data.header.mins

array([ 1.2598e+05,  4.8373e+05, -5.4670e+00])

In [7]:
min_x, max_x = round(las_data.x.min()), round(las_data.x.max())
min_y, max_y = round(las_data.y.min()), round(las_data.y.max())

# Define size of the region
x_length = max_x - min_x
y_length = max_y - min_y

In [14]:
veg_points.xyz

array([[1.25999463e+05, 4.83730062e+05, 1.37900000e+00],
       [1.25999147e+05, 4.83730237e+05, 1.41600000e+00],
       [1.25999136e+05, 4.83730003e+05, 1.42600000e+00],
       ...,
       [1.27002546e+05, 4.85014363e+05, 1.89900000e+00],
       [1.27002497e+05, 4.85014584e+05, 2.21000000e+00],
       [1.27002423e+05, 4.85014274e+05, 2.04800000e+00]])

In [15]:
points_list = np.array(veg_points.xyz)

In [17]:
import startinpy

In [18]:
dt = startinpy.DT()

In [19]:
dt.insert(veg_points.xyz, "BBox")

In [76]:
def interpolation_vegetation(LasData, veg_points, resolution, dt, no_data_value=-9999):
    """
    Create a vegetation raster using Inverse Distance Weighting (IDW) interpolation.

    Parameters:
    - LasData (laspy.LasData): Input LAS data.
    - points (laspy.LasData): Vegetation points to be interpolated.
    - center_point (tuple): Center coordinates (x, y) of the region of interest.
    - resolution (float): Resolution of the raster.
    - half_size (float): Half of the size of the tile (defines the extent of the raster).
    - search_radius (float): Radius for the IDW search. Default is 1.
    - power (float): Power parameter for the IDW algorithm. Default is 2.

    Returns:
    - vege_raster (np.ndarray): Generated raster for vegetation.
    - grid_center_xy (tuple): Grid of x, y center coordinates for each raster cell.
    """

    # Extents of the pc
    min_x, max_x = round(LasData.x.min()), round(LasData.x.max())
    min_y, max_y = round(LasData.y.min()), round(LasData.y.max())

    # Define size of the region
    x_length = max_x - min_x
    y_length = max_y - min_y

    # Number of rows and columns
    cols = round(x_length / resolution)
    rows = round(y_length / resolution)

    # Initialize raster grid
    vege_raster = np.full((rows, cols), no_data_value, dtype=np.float32)

    # Calculate center coords for each grid cell
    grid_center_xy = functions.raster_center_coords(min_x, max_x, min_y, max_y, resolution)
    


    if dt.number_of_vertices == 0:
        print(" There are no vegetation points in the current area.")
        return vege_raster, grid_center_xy
    
    # Flatten the grid to get a list of all (col, row) locations
    locs = np.column_stack((grid_center_xy[0].ravel(), grid_center_xy[1].ravel()))
    
    vegetation_points = np.column_stack((veg_points.x, veg_points.y))
    tree = cKDTree(vegetation_points)

    # Find the distance to the nearest vegetation point for each grid cell
    distances, _ = tree.query(locs, k=1)  # Find nearest neighbor distances
    
    distance_threshold = 1
    # Mask cells where the distance to the nearest point exceeds the threshold
    within_threshold_mask = distances <= distance_threshold

    # Perform interpolation only for locations within the distance threshold
    valid_locs = locs[within_threshold_mask]

    # laplace interpolation
    interpolated_values = dt.interpolate({"method": "Laplace"}, valid_locs)

    # reshape interpolated grid back to og
    interpolated_grid = np.full_like(vege_raster, no_data_value, dtype=np.float32)  # Start with no_data
    interpolated_grid.ravel()[within_threshold_mask] = interpolated_values 

    return interpolated_grid, grid_center_xy

In [74]:
# vegetation_data = interpolation_vegetation(las_data, veg_points, 0.5, dt3)

In [67]:
# vegetation_data[0]

array([[-9.99900000e+03, -9.99900000e+03, -9.99900000e+03, ...,
         1.14863224e-01, -4.00468677e-01,             nan],
       [-9.99900000e+03, -9.99900000e+03, -9.99900000e+03, ...,
        -1.55634314e-01, -1.87330991e-01,             nan],
       [-9.99900000e+03, -9.99900000e+03, -9.99900000e+03, ...,
        -9.99900000e+03, -9.99900000e+03, -9.99900000e+03],
       ...,
       [ 4.44815934e-01,  8.68079290e-02,  1.89682758e+00, ...,
        -9.99900000e+03,             nan,             nan],
       [ 2.31232214e+00, -5.39936088e-02,  7.85766006e-01, ...,
        -9.99900000e+03, -9.99900000e+03, -9.99900000e+03],
       [            nan,  1.97609425e+00,  1.22713971e+00, ...,
        -9.99900000e+03, -9.99900000e+03, -9.99900000e+03]], dtype=float32)

In [3]:
def chm_creation(LasData, vegetation_data, output_filename, resolution=0.5):

    veg_raster = vegetation_data[0]
    grid_centers = vegetation_data[1]
    top_left_x = grid_centers[0][0, 0] - resolution / 2
    top_left_y = grid_centers[1][0, 0] + resolution / 2

    transform = functions.create_affine_transform(top_left_x, top_left_y, resolution)

    functions.write_output(veg_points, veg_raster, transform, output_filename, True)

In [75]:
chm_creation(las_data, vegetation_data, "tryout/chm5.TIF")

File written to 'tryout/chm5.TIF'


In [5]:
save_vegetation_points_as_las(las_data, output_las, veg_points, ndvi_threshold=0.0)

Vegetation points saved to data/output_25gn224.las


In [4]:
import functions
import os
import laspy
import time  # Import time module to track time
from tqdm import tqdm
import numpy as np

def extract_vegetation_points(LasData, ndvi_threshold=0.1):
    """
    Extract vegetation points based on classification and NDVI threshold.

    Parameters:
    - LasData (laspy.LasData): Input point cloud data.
    - ndvi_threshold (float): The NDVI threshold to classify vegetation.

    Returns:
    - veg_points (laspy.LasData): Filtered vegetation points as a new LasData object.
    """

    # Filter points based on classification (vegetation-related classes)
    possible_vegetation_points = LasData[(LasData.classification == 1) |  # Unclassified
                                         (LasData.classification == 3) |  # Low vegetation
                                         (LasData.classification == 4) |  # Medium vegetation
                                         (LasData.classification == 5)]  # High vegetation

    # Calculate NDVI
    red = possible_vegetation_points.red
    nir = possible_vegetation_points.nir
    ndvi = (nir.astype(float) - red) / (nir + red)

    # Filter the points whose NDVI is greater than the threshold
    veg_points = possible_vegetation_points[ndvi > ndvi_threshold]
    print("extracted vegetation points")
    
      # EDIT THIS, ONLY FOR THE TILE WITH ISSUES (25GN224LAZ)
    heights =veg_points.z
    min_height = heights.min()
    
    # Filter out points with heights between the minimum height and 1.5 meters
    filtered_veg_points = veg_points[(heights <= min_height) | (heights > 1.5)]

    return filtered_veg_points

def chm_creation(LasData, vegetation_data, output_filename, resolution=0.5):

    veg_raster = vegetation_data[0]
    grid_centers = vegetation_data[1]
    top_left_x = grid_centers[0][0, 0] - resolution / 2
    top_left_y = grid_centers[1][0, 0] + resolution / 2

    transform = functions.create_affine_transform(top_left_x, top_left_y, resolution)

    functions.write_output(LasData, veg_raster, transform, output_filename, True)

def interpolation_vegetation(LasData, veg_points, resolution, no_data_value=-9999):
    """
    Create a vegetation raster using Inverse Distance Weighting (IDW) interpolation.

    Parameters:
    - LasData (laspy.LasData): Input LAS data.
    - veg_points (laspy.LasData): Vegetation points to be interpolated.
    - resolution (float): Resolution of the raster.
    - no_data_value (int): Value for no data

    Returns:
    - interpolated_grid (np.ndarray): Generated raster for vegetation.
    - grid_center_xy (tuple): Grid of x, y center coordinates for each raster cell.
    """

    # Extents of the pc
    min_x, max_x = round(LasData.x.min()), round(LasData.x.max())
    min_y, max_y = round(LasData.y.min()), round(LasData.y.max())

    # Define size of the region
    x_length = max_x - min_x
    y_length = max_y - min_y

    # Number of rows and columns
    cols = round(x_length / resolution)
    rows = round(y_length / resolution)

    # Initialize raster grid
    vege_raster = np.full((rows, cols), no_data_value, dtype=np.float32)

    # Calculate center coords for each grid cell
    grid_center_xy = functions.raster_center_coords(min_x, max_x, min_y, max_y, resolution)
    
    if  veg_points.x.shape[0] == 0:
        print("There are no vegetation points in the current area.")
        vege_raster = np.full((rows, cols), 0, dtype=np.float32)
        return vege_raster, grid_center_xy
    
    # create the delaunay triangulation
    dt = startinpy.DT()
    dt.insert(veg_points.xyz, "BBox")
    
    print("finished DT")
    
    # Flatten the grid to get a list of all center coords
    locs = np.column_stack((grid_center_xy[0].ravel(), grid_center_xy[1].ravel()))
    
    vegetation_points = np.column_stack((veg_points.x, veg_points.y))
    tree = cKDTree(vegetation_points)

    # Find the distance to the nearest vegetation point for each grid cell
    distances, _ = tree.query(locs, k=1) 
    
    distance_threshold = 1
    # masking cells that exceed threshold
    within_threshold_mask = distances <= distance_threshold
    print("masked")
    # Interpolation only for those near
    valid_locs = locs[within_threshold_mask]

    # laplace interpolation
    interpolated_values = dt.interpolate({"method": "Laplace"}, valid_locs)

    # reshape interpolated grid back to og
    interpolated_grid = np.full_like(vege_raster, no_data_value, dtype=np.float32)  # Start with no_data
    interpolated_grid.ravel()[within_threshold_mask] = interpolated_values 
    print("interpolated")

    return interpolated_grid, grid_center_xy


def process_laz_files(input_folder, output_folder, ndvi_threshold=0.0, resolution=0.5):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    laz_files = [f for f in os.listdir(input_folder) if f.endswith(".LAZ")]

    total_start_time = time.time()

    # Iterate over files
    for file_name in tqdm(laz_files, desc="Processing files", unit="file"):
        file_path = os.path.join(input_folder, file_name)
        tile_name = os.path.splitext(file_name)[0]
        output_filename = os.path.join(output_folder, f"CHM_{tile_name}.TIF")
        print(f"processing tile {tile_name}")
        start_time = time.time()

        # Load LAS data
        with laspy.open(file_path) as las:
            LasData = las.read()

        # Extract vegetation points
        veg_points = extract_vegetation_points(LasData, ndvi_threshold=ndvi_threshold)
        
        vegetation_data = interpolation_vegetation(LasData, veg_points, 0.5)

        # Create the CHM and save it
        chm_creation(LasData, vegetation_data, output_filename, resolution=resolution)

        # Calculate time taken for current file
        elapsed_time = time.time() - start_time
        print(f"\nProcessed {file_name} in {elapsed_time:.2f} seconds and saved output to {output_filename}")

        # os.remove(file_path)
        # print(f"Deleted file: {file_path}")

    # Calculate and print total processing time
    total_elapsed_time = time.time() - total_start_time
    print(f"\nAll files processed in {total_elapsed_time:.2f} seconds.")

In [4]:
input_folder = "C:/Geomatics/data/FORGOTTENLAZ/25BZ2"
output_folder = "C:\Geomatics\data\output_forgotten"
process_laz_files(input_folder, output_folder)

Processing files:   0%|          | 0/2 [00:00<?, ?file/s]

processing tile 25BZ2_09
extracted vegetation points
finished DT
masked


Processing files:  50%|█████     | 1/2 [00:32<00:32, 32.79s/file]

interpolated
File written to 'C:\Geomatics\data\output_forgotten\CHM_25BZ2_09.TIF'

Processed 25BZ2_09.LAZ in 32.79 seconds and saved output to C:\Geomatics\data\output_forgotten\CHM_25BZ2_09.TIF
processing tile 25BZ2_16
extracted vegetation points
finished DT
masked


Processing files: 100%|██████████| 2/2 [01:24<00:00, 42.11s/file]

interpolated
File written to 'C:\Geomatics\data\output_forgotten\CHM_25BZ2_16.TIF'

Processed 25BZ2_16.LAZ in 51.44 seconds and saved output to C:\Geomatics\data\output_forgotten\CHM_25BZ2_16.TIF

All files processed in 84.25 seconds.


In [5]:
input_folder = "C:/Geomatics/data/FORGOTTENLAZ/25GZ1"
output_folder = "C:\Geomatics\data\output_forgotten"
process_laz_files(input_folder, output_folder)

Processing files:   0%|          | 0/1 [00:00<?, ?file/s]

processing tile 25GZ1_20
extracted vegetation points
finished DT
masked


Processing files: 100%|██████████| 1/1 [00:17<00:00, 17.69s/file]

interpolated
File written to 'C:\Geomatics\data\output_forgotten\CHM_25GZ1_20.TIF'

Processed 25GZ1_20.LAZ in 17.69 seconds and saved output to C:\Geomatics\data\output_forgotten\CHM_25GZ1_20.TIF

All files processed in 17.70 seconds.


In [6]:
input_folder = "C:/Geomatics/data/FORGOTTENLAZ/25DN2"
output_folder = "C:\Geomatics\data\output_forgotten"
process_laz_files(input_folder, output_folder)

Processing files:   0%|          | 0/2 [00:00<?, ?file/s]

processing tile 25DN2_22
extracted vegetation points
finished DT
masked


Processing files:  50%|█████     | 1/2 [02:25<02:25, 145.20s/file]

interpolated
File written to 'C:\Geomatics\data\output_forgotten\CHM_25DN2_22.TIF'

Processed 25DN2_22.LAZ in 145.20 seconds and saved output to C:\Geomatics\data\output_forgotten\CHM_25DN2_22.TIF
processing tile 25DN2_23
extracted vegetation points
finished DT
masked


Processing files: 100%|██████████| 2/2 [05:56<00:00, 178.41s/file]

interpolated
File written to 'C:\Geomatics\data\output_forgotten\CHM_25DN2_23.TIF'

Processed 25DN2_23.LAZ in 211.60 seconds and saved output to C:\Geomatics\data\output_forgotten\CHM_25DN2_23.TIF

All files processed in 356.82 seconds.


In [7]:
input_folder = "C:/Geomatics/data/FORGOTTENLAZ/25HN1"
output_folder = "C:\Geomatics\data\output_forgotten"
process_laz_files(input_folder, output_folder)

Processing files:   0%|          | 0/1 [00:00<?, ?file/s]

processing tile 25HN1_24
extracted vegetation points
finished DT
masked


Processing files: 100%|██████████| 1/1 [00:44<00:00, 44.50s/file]

interpolated
File written to 'C:\Geomatics\data\output_forgotten\CHM_25HN1_24.TIF'

Processed 25HN1_24.LAZ in 44.50 seconds and saved output to C:\Geomatics\data\output_forgotten\CHM_25HN1_24.TIF

All files processed in 44.51 seconds.


In [5]:
input_folder = "C:/Geomatics/data/FORGOTTENLAZ/25GN2"
output_folder = "C:\Geomatics\data\output_forgotten"
process_laz_files(input_folder, output_folder)

Processing files:   0%|          | 0/1 [00:00<?, ?file/s]

processing tile 25GN2_24
extracted vegetation points
finished DT
masked
interpolated


Processing files: 100%|██████████| 1/1 [07:29<00:00, 449.69s/file]

File written to 'C:\Geomatics\data\output_forgotten\CHM_25GN2_24.TIF'

Processed 25GN2_24.LAZ in 449.67 seconds and saved output to C:\Geomatics\data\output_forgotten\CHM_25GN2_24.TIF

All files processed in 449.71 seconds.


In [29]:
input_folder = "D:/Geomatics/LAZ_TILES/25HN1_special"
output_folder = "D:/Geomatics/CHM/25HN1_special"
process_laz_files(input_folder, output_folder)

Processing files:   0%|          | 0/2 [00:00<?, ?file/s]

processing tile 25HN1_07
extracted vegetation points


Processing files:  50%|█████     | 1/2 [00:02<00:02,  2.42s/file]

There are no vegetation points in the current area.
File written to 'D:/Geomatics/CHM/25HN1_special\CHM_25HN1_07.TIF'

Processed 25HN1_07.LAZ in 2.42 seconds and saved output to D:/Geomatics/CHM/25HN1_special\CHM_25HN1_07.TIF
processing tile 25HN1_06
extracted vegetation points


Processing files: 100%|██████████| 2/2 [00:09<00:00,  4.69s/file]

There are no vegetation points in the current area.
File written to 'D:/Geomatics/CHM/25HN1_special\CHM_25HN1_06.TIF'

Processed 25HN1_06.LAZ in 6.97 seconds and saved output to D:/Geomatics/CHM/25HN1_special\CHM_25HN1_06.TIF

All files processed in 9.39 seconds.


In [ ]:
input_folder = "D:/Geomatics/LAZ_TILES/25HZ2"
output_folder = "D:/Geomatics/CHM/25HZ2"
process_laz_files(input_folder, output_folder)

Processing files:   0%|          | 0/3 [00:00<?, ?file/s]

In [ ]:
input_folder = "D:/Geomatics/LAZ_TILES/25BZ2"
output_folder = "D:/Geomatics/CHM/25BZ2"
process_laz_files(input_folder, output_folder)

In [ ]:
input_folder = "D:/Geomatics/LAZ_TILES/25BZ1"
output_folder = "D:/Geomatics/CHM/25BZ1"
process_laz_files(input_folder, output_folder)

In [ ]:
input_folder = "D:/Geomatics/LAZ_TILES/25DN1"
output_folder = "D:/Geomatics/CHM/25DN1"
process_laz_files(input_folder, output_folder)

In [ ]:
input_folder = "D:/Geomatics/LAZ_TILES/25DN2"
output_folder = "D:/Geomatics/CHM/25DN2"
process_laz_files(input_folder, output_folder)

In [ ]:
input_folder = "D:/Geomatics/LAZ_TILES/25EZ1"
output_folder = "D:/Geomatics/CHM/25EZ1"
process_laz_files(input_folder, output_folder)

In [ ]:
input_folder = "D:/Geomatics/LAZ_TILES/25EZ2"
output_folder = "D:/Geomatics/CHM/25EZ2"
process_laz_files(input_folder, output_folder)

In [ ]:
input_folder = "D:/Geomatics/LAZ_TILES/25FZ1"
output_folder = "D:/Geomatics/CHM/25FZ1"
process_laz_files(input_folder, output_folder)

In [ ]:
input_folder = "D:/Geomatics/LAZ_TILES/25GN1"
output_folder = "D:/Geomatics/CHM/25GN1"
process_laz_files(input_folder, output_folder)

In [3]:
input_folder = "D:/Geomatics/LAZ_TILES/25GN2"
output_folder = "D:/Geomatics/CHM/25GN2"
process_laz_files(input_folder, output_folder)

Processing files:   0%|          | 0/22 [00:00<?, ?file/s]

processing tile 25GN2_23
extracted vegetation points
finished DT
masked
interpolated


Processing files:   5%|▍         | 1/22 [04:20<1:31:14, 260.69s/file]

File written to 'D:/Geomatics/CHM/25GN2\CHM_25GN2_23.TIF'

Processed 25GN2_23.LAZ in 260.69 seconds and saved output to D:/Geomatics/CHM/25GN2\CHM_25GN2_23.TIF
processing tile 25GN2_22
extracted vegetation points
finished DT
masked


Processing files:   9%|▉         | 2/22 [05:35<50:29, 151.48s/file]  

...

File written to 'D:/Geomatics/CHM/25GN2\CHM_25GN2_04.TIF'

Processed 25GN2_04.LAZ in 100.67 seconds and saved output to D:/Geomatics/CHM/25GN2\CHM_25GN2_04.TIF
processing tile 25GN2_03
extracted vegetation points
finished DT
masked
interpolated


Processing files:  95%|█████████▌| 21/22 [37:47<01:24, 84.02s/file]

File written to 'D:/Geomatics/CHM/25GN2\CHM_25GN2_03.TIF'

Processed 25GN2_03.LAZ in 92.86 seconds and saved output to D:/Geomatics/CHM/25GN2\CHM_25GN2_03.TIF
processing tile 25GN2_02
extracted vegetation points
finished DT
masked


Processing files: 100%|██████████| 22/22 [38:20<00:00, 104.57s/file]

interpolated
File written to 'D:/Geomatics/CHM/25GN2\CHM_25GN2_02.TIF'

Processed 25GN2_02.LAZ in 32.63 seconds and saved output to D:/Geomatics/CHM/25GN2\CHM_25GN2_02.TIF

All files processed in 2300.46 seconds.


In [ ]:
input_folder = "D:/Geomatics/LAZ_TILES/25GZ1"
output_folder = "D:/Geomatics/CHM/25GZ1"
process_laz_files(input_folder, output_folder)

In [ ]:
input_folder = "D:/Geomatics/LAZ_TILES/25GZ2"
output_folder = "D:/Geomatics/CHM/25GZ2"
process_laz_files(input_folder, output_folder)

In [ ]:
input_folder = "D:/Geomatics/LAZ_TILES/25HN1"
output_folder = "D:/Geomatics/CHM/25HN1"
process_laz_files(input_folder, output_folder)

In [ ]:
input_folder = "D:/Geomatics/LAZ_TILES/25HZ1"
output_folder = "D:/Geomatics/CHM/25HZ1"
process_laz_files(input_folder, output_folder)

In [ ]:
input_folder = "D:/Geomatics/LAZ_TILES/25HZ2"
output_folder = "D:/Geomatics/CHM/25HZ2"
process_laz_files(input_folder, output_folder)